# Proyecto: Spaceflights

In [24]:
from typing import Dict, Sequence, Tuple

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

## Procesamiento de datos

In [2]:
companies = pd.read_csv("companies.csv")
shuttles = pd.read_excel("shuttles.xlsx")
reviews = pd.read_csv("reviews.csv")

*Companies*

In [3]:
print(companies.info())
companies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 77096 non-null  int64  
 1   company_rating     47187 non-null  object 
 2   company_location   57966 non-null  object 
 3   total_fleet_count  77089 non-null  float64
 4   iata_approved      77089 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.9+ MB
None


,id,company_rating,company_location,total_fleet_count,iata_approved
0,35029,100%,Niue,4.0,f
1,30292,67%,Anguilla,6.0,f
2,19032,67%,Russian Federation,4.0,f
3,8238,91%,Barbados,15.0,t
4,30342,NaN,Sao Tome and Principe,2.0,t


In [4]:
def _is_true(x):
    return x == "t"


def _parse_percentage(x):
    x = x.str.replace("%", "", regex=False)
    x = x.astype(float) / 100
    return x


def preprocess_companies(companies: pd.DataFrame) -> pd.DataFrame:
    work_df = companies.copy()
    work_df["iata_approved"] = _is_true(companies["iata_approved"])
    work_df["company_rating"] = _parse_percentage(companies["company_rating"])
    return work_df


preprocessed_companies = preprocess_companies(companies=companies)
preprocessed_companies

,id,company_rating,company_location,total_fleet_count,iata_approved
0,35029,1.00,Niue,4.0,False
1,30292,0.67,Anguilla,6.0,False
2,19032,0.67,Russian Federation,4.0,False
3,8238,0.91,Barbados,15.0,True
4,30342,NaN,Sao Tome and Principe,2.0,True
...,...,...,...,...,...
77091,6654,1.00,Tonga,3.0,False
77092,8000,NaN,Chile,2.0,True
77093,14296,NaN,Netherlands,4.0,False
77094,27363,0.80,NaN,3.0,True


*Shuttles*

In [5]:
print(shuttles.info())
shuttles.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       77096 non-null  int64  
 1   shuttle_location         77096 non-null  object 
 2   shuttle_type             77096 non-null  object 
 3   engine_type              77096 non-null  object 
 4   engine_vendor            77096 non-null  object 
 5   engines                  77057 non-null  float64
 6   passenger_capacity       77096 non-null  int64  
 7   cancellation_policy      77096 non-null  object 
 8   crew                     76947 non-null  float64
 9   d_check_complete         77096 non-null  object 
 10  moon_clearance_complete  77096 non-null  object 
 11  price                    77096 non-null  object 
 12  company_id               77096 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 7.6+ MB
None


,id,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,moon_clearance_complete,price,company_id
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,f,f,"$1,325.0",35029
1,36260,Anguilla,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,t,f,"$1,780.0",30292
2,57015,Russian Federation,Type V5,Quantum,ThetaBase Services,1.0,2,moderate,0.0,f,f,"$1,715.0",19032
3,14035,Barbados,Type V5,Plasma,ThetaBase Services,3.0,6,strict,3.0,f,f,"$4,770.0",8238
4,10036,Sao Tome and Principe,Type V2,Plasma,ThetaBase Services,2.0,4,strict,2.0,f,f,"$2,820.0",30342


In [6]:
def _parse_money(x):
    x = x.str.replace("$", "", regex=False).str.replace(",", "", regex=False)
    x = x.astype(float)
    return x


def preprocess_shuttles(shuttles: pd.DataFrame) -> pd.DataFrame:
    work_df = shuttles.copy()
    work_df["d_check_complete"] = _is_true(shuttles["d_check_complete"])
    work_df["moon_clearance_complete"] = _is_true(shuttles["moon_clearance_complete"])
    work_df["price"] = _parse_money(shuttles["price"])
    return work_df


preprocessed_shuttles = preprocess_shuttles(shuttles=shuttles)
preprocessed_shuttles

,id,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,moon_clearance_complete,price,company_id
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,False,1325.0,35029
1,36260,Anguilla,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,True,False,1780.0,30292
2,57015,Russian Federation,Type V5,Quantum,ThetaBase Services,1.0,2,moderate,0.0,False,False,1715.0,19032
3,14035,Barbados,Type V5,Plasma,ThetaBase Services,3.0,6,strict,3.0,False,False,4770.0,8238
4,10036,Sao Tome and Principe,Type V2,Plasma,ThetaBase Services,2.0,4,strict,2.0,False,False,2820.0,30342
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77091,4368,Barbados,Type V5,Quantum,ThetaBase Services,2.0,4,flexible,2.0,True,False,4107.0,6654
77092,2983,Bouvet Island (Bouvetoya),Type F5,Quantum,ThetaBase Services,1.0,1,flexible,1.0,True,False,1169.0,8000
77093,69684,Micronesia,Type V5,Plasma,ThetaBase Services,0.0,2,flexible,1.0,True,False,1910.0,14296
77094,21738,Uzbekistan,Type V5,Plasma,ThetaBase Services,1.0,2,flexible,1.0,True,False,2170.0,27363


*Reviews*

In [7]:
print(reviews.info())
reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   shuttle_id               77096 non-null  int64  
 1   review_scores_rating     55956 non-null  float64
 2   review_scores_comfort    55896 non-null  float64
 3   review_scores_amenities  55909 non-null  float64
 4   review_scores_trip       55833 non-null  float64
 5   review_scores_crew       55902 non-null  float64
 6   review_scores_location   55831 non-null  float64
 7   review_scores_price      55828 non-null  float64
 8   number_of_reviews        77096 non-null  int64  
 9   reviews_per_month        57553 non-null  float64
dtypes: float64(8), int64(2)
memory usage: 5.9 MB
None


,shuttle_id,review_scores_rating,review_scores_comfort,review_scores_amenities,review_scores_trip,review_scores_crew,review_scores_location,review_scores_price,number_of_reviews,reviews_per_month
0,63561,97.0,10.0,9.0,10.0,10.0,9.0,10.0,133,1.65
1,36260,90.0,8.0,9.0,10.0,9.0,9.0,9.0,3,0.09
2,57015,95.0,9.0,10.0,9.0,10.0,9.0,9.0,14,0.14
3,14035,93.0,10.0,9.0,9.0,9.0,10.0,9.0,39,0.42
4,10036,98.0,10.0,10.0,10.0,10.0,9.0,9.0,92,0.94


*Uniendo y separando*

In [21]:
def create_model_input_table(
    shuttles: pd.DataFrame, companies: pd.DataFrame, reviews: pd.DataFrame
) -> pd.DataFrame:
    rated_shuttles = shuttles.merge(reviews, left_on="id", right_on="shuttle_id")
    model_input_table = rated_shuttles.merge(
        companies, left_on="company_id", right_on="id"
    )
    model_input_table = model_input_table.dropna()
    return model_input_table


model_input_table = create_model_input_table(
    shuttles=preprocessed_shuttles, companies=preprocessed_companies, reviews=reviews
)
model_input_table.head()

,id_x,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,...,review_scores_crew,review_scores_location,review_scores_price,number_of_reviews,reviews_per_month,id_y,company_rating,company_location,total_fleet_count,iata_approved
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
1,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
2,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
3,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
4,53260,Niue,Type V5,Quantum,"Banks, Wood and Phillips",1.0,2,strict,1.0,False,...,10.0,9.0,10.0,37,0.48,35029,1.0,Niue,4.0,False


## Ciencia de datos

*Uniendo y separando*

In [27]:
def split_data(
    data: pd.DataFrame, test_size: float, random_state: int
) -> Tuple:
    X = data[[
        "engines",
        "passenger_capacity",
        "crew",
        "d_check_complete",
        "moon_clearance_complete",
        "iata_approved",
        "company_rating",
        "review_scores_rating",
    ]]
    y = data["price"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = split_data(
    data=model_input_table, test_size=0.2, random_state=3
)

*Entrenamiento y evaluación*

In [28]:
def train_model(X_train: pd.DataFrame, y_train: pd.Series) -> LinearRegression:
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    return regressor


trained_model = train_model(X_train=X_train, y_train=y_train)

In [29]:
def evaluate_model(regressor: LinearRegression, X_test: np.ndarray, y_test: np.ndarray):
    y_pred = regressor.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f"Model has a coefficient R^2 of {score:.3f}.")


evaluate_model(regressor=trained_model, X_test=X_test, y_test=y_test)

Model has a coefficient R^2 of 0.462.
